In [ ]:
!pip install "gluonts[mxnet,pro]"

In [2]:
import pandas as pd
from gluonts.dataset.common import ListDataset
from gluonts.mx.model.deepar import DeepAREstimator
from gluonts.mx.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator
import matplotlib.pyplot as plt

In [3]:
from google.colab import files
uploaded = files.upload()

Saving CPI_data.csv to CPI_data.csv


In [4]:
# read in the time series data
data = pd.read_csv("CPI_data.csv", index_col="date", parse_dates=True)

# split data into training and validation sets
train_data = data.iloc[:456]
val_data = data.iloc[456:]

# create a GluonTS ListDataset object for training data
training_data = ListDataset(
    [{"start": train_data.index[0], "target": train_data["CPI"].values}],
    freq="M"
)

In [5]:
# define the DeepAR estimator object
estimator = DeepAREstimator(
    freq="M",
    prediction_length=24,
    context_length=12,
    num_layers=2,
    num_cells=40,
    trainer=Trainer(epochs=50)
)

In [6]:
# train the model on the training data
predictor = estimator.train(training_data=training_data)

100%|██████████| 50/50 [00:04<00:00, 10.35it/s, epoch=50/50, avg_epoch_loss=2.25]


In [7]:
# create a GluonTS ListDataset object for validation data
validation_data = ListDataset(
    [{"start": val_data.index[0], "target": val_data["CPI"].values}],
    freq="M"
)

# make predictions on the validation data
forecast_it, ts_it = make_evaluation_predictions(
    dataset=validation_data, predictor=predictor, num_samples=100
)

In [8]:
forecast = next(forecast_it)

# extract the predicted values from the forecast
forecast_values = forecast.mean.tolist()

forecast_values

[251.90402221679688,
 254.01748657226562,
 255.21290588378906,
 256.9194030761719,
 258.6391906738281,
 260.33123779296875,
 261.40496826171875,
 262.11492919921875,
 263.1888122558594,
 263.2613220214844,
 263.3153991699219,
 264.358642578125,
 260.1767272949219,
 259.1431884765625,
 259.26239013671875,
 259.4150390625,
 259.9247131347656,
 260.5379638671875,
 261.4583435058594,
 261.7754821777344,
 262.11077880859375,
 262.2750549316406,
 261.8799133300781,
 259.1002197265625]

In [11]:
import pickle

# save the trained model to a file
with open('trained_deepar_model.pkl', 'wb') as f:
    pickle.dump(predictor, f)

In [12]:
files.download('trained_deepar_model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# forects = list(forecast_it)

# get the forecasted values and print them in an array
# forecast_values = forects[0].samples.mean(axis=0)
# print(forecast_values)